In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn.ensemble import RandomForestClassifier
import time
import xgboost as xgb
import lightgbm as lgb
import warnings
import pickle

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use(style='seaborn')
%matplotlib inline

# Data Merging

In [2]:
%%time
train_id = pd.read_csv('train_identity.csv')
train_tr = pd.read_csv('train_transaction.csv')
test_id = pd.read_csv('test_identity.csv')
test_tr = pd.read_csv('test_transaction.csv')

train = pd.merge(train_tr, train_id, how='left', 
                 on='TransactionID', left_index=True, 
                right_index=True)

test = pd.merge(test_tr, test_id, how='left',
               on='TransactionID', left_index=True,
               right_index=True)

del train_tr,test_id,train_id,test_tr

Wall time: 1min 32s


In [3]:
print(train.shape)
print(test.shape)

(590540, 434)
(506691, 433)


In [4]:
#for model training
target = train['isFraud']

# Data cleaning

In [5]:
train.columns

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=434)

In [6]:
test.columns

Index(['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD',
       'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'id-31', 'id-32', 'id-33', 'id-34', 'id-35', 'id-36', 'id-37', 'id-38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=433)

In [7]:
# train and test column names supposes to be same but no for this dataset
# we have to fix them.
# id-01 supposed to be id_31
def differentcolumns(traincols, testcols):
    for i in traincols:
        if i not in testcols:
            print(i)
differentcolumns(train.columns, test.columns)

isFraud
id_01
id_02
id_03
id_04
id_05
id_06
id_07
id_08
id_09
id_10
id_11
id_12
id_13
id_14
id_15
id_16
id_17
id_18
id_19
id_20
id_21
id_22
id_23
id_24
id_25
id_26
id_27
id_28
id_29
id_30
id_31
id_32
id_33
id_34
id_35
id_36
id_37
id_38


In [8]:
test = test.rename(columns={"id-01": "id_01", "id-02": "id_02", "id-03": "id_03", 
                            "id-06": "id_06", "id-05": "id_05", "id-04": "id_04", 
                            "id-07": "id_07", "id-08": "id_08", "id-09": "id_09", 
                            "id-10": "id_10", "id-11": "id_11", "id-12": "id_12", 
                            "id-15": "id_15", "id-14": "id_14", "id-13": "id_13", 
                            "id-16": "id_16", "id-17": "id_17", "id-18": "id_18", 
                            "id-21": "id_21", "id-20": "id_20", "id-19": "id_19", 
                            "id-22": "id_22", "id-23": "id_23", "id-24": "id_24", 
                            "id-27": "id_27", "id-26": "id_26", "id-25": "id_25", 
                            "id-28": "id_28", "id-29": "id_29", "id-30": "id_30", 
                            "id-31": "id_31", "id-32": "id_32", "id-33": "id_33", 
                            "id-34": "id_34", "id-35": "id_35", "id-36": "id_36", 
                            "id-37": "id_37", "id-38": "id_38"})

In [9]:
test.columns

Index(['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD',
       'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=433)

## Fixing the missing values

In [10]:
train.isnull().sum().sort_values(ascending=False)

id_24            585793
id_25            585408
id_07            585385
id_08            585385
id_21            585381
                  ...  
C11                   0
C12                   0
C13                   0
C14                   0
TransactionID         0
Length: 434, dtype: int64

In [11]:
# Making a dataframe about total missing values and percentage of a column

def getnulls(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (data.isnull().sum() / len(data)).sort_values(ascending=False)
    missing_data = pd.concat([total, percent],  keys=['total', 'precent'],axis=1)
    return missing_data

# Train Data Missing Values
missing_data_train = getnulls(train)
missing_data_train.head(len(train)).T

,id_24,id_25,id_07,id_08,id_21,id_26,id_22,id_23,id_27,dist2,D7,id_18,D13,D14,D12,id_04,id_03,D6,id_33,id_10,D8,D9,id_09,id_30,id_32,id_34,id_14,V149,V153,V156,V154,V155,V157,V158,V161,V163,V147,V148,V162,V146,V138,V139,V140,V141,V142,V166,V164,V160,V159,V143,V144,V145,V152,V151,V150,V165,V339,V322,V338,V337,V336,V335,V334,V333,V332,V331,V330,V329,V328,V327,V326,V325,V324,V323,DeviceInfo,id_13,id_16,V237,V230,V267,V266,V265,V264,V263,V262,V261,V260,V228,V258,V257,V229,V254,V269,V253,V252,V231,V232,V249,V248,V247,V246,V233,V244,V243,V242,V241,V240,V268,V226,V218,V278,V217,V236,V219,V223,V224,V235,V277,V275,V274,V273,V225,V276,id_05,id_06,R_emaildomain,id_20,id_19,id_17,V202,V207,V204,V203,V214,V211,V212,V213,V215,V216,V199,V168,V191,V206,V187,V182,V183,V178,V196,V177,V176,V167,V186,V181,V179,V190,V205,V192,V193,V173,V172,V174,V171,V175,V197,V180,V201,V184,V185,V188,V189,V194,V198,V195,V200,V170,V208,V209,V210,V169,id_31,DeviceType,id_02,id_11,id_28,id_29,id_38,id_35,id_36,id_37,id_15,V272,V271,V270,V239,V220,V221,V222,V234,V238,V259,V227,V245,V250,V251,V255,V256,id_12,id_01,dist1,M5,M7,M8,M9,D5,M4,D2,V5,V1,V2,V3,V4,V6,D11,V7,V8,V9,V10,V11,M3,M2,M1,D3,M6,V42,V52,V51,V50,V48,V47,V46,V45,V44,V43,V49,V41,V40,V39,V38,V37,V36,V35,D4,P_emaildomain,V76,V77,V78,V79,V83,V80,V81,V82,V90,V94,V93,V92,V91,V89,V88,V87,V86,V75,V85,V84,D15,V71,V53,V54,V55,V56,V57,V58,V59,V60,V62,V72,V63,V64,V65,V66,V67,V68,V69,V74,V70,V61,V73,V18,V14,V27,V30,V26,V25,V24,V23,V22,V21,V20,V19,V31,V17,V16,V15,V13,V29,V12,V32,V33,V34,V28,D10,addr2,addr1,card2,card5,card4,card6,card3,V314,V313,V283,D1,V282,V288,V289,V315,V296,V300,V301,V281,V95,V96,V106,V114,V113,V112,V111,V110,V109,V108,V107,V105,V137,V104,V103,V102,V101,V100,V99,V98,V97,V115,V116,V117,V118,V136,V135,V134,V133,V132,V131,V130,V129,V128,V127,V126,V124,V123,V122,V121,V120,V119,V125,V304,V303,V280,V287,V292,V293,V294,V295,V297,V298,V299,V302,V321,V286,V291,V284,V285,V279,V305,V320,V319,V318,V317,V316,V312,V311,V310,V309,V308,V307,V306,V290,card1,ProductCD,TransactionAmt,TransactionDT,isFraud,C9,C1,C2,C3,C4,C5,C6,C7,C8,C10,C11,C12,C13,C14,TransactionID
total,585793.000000,585408.00000,585385.000000,585385.000000,585381.000000,585377.000000,585371.000000,585371.000000,585371.000000,552913.000000,551623.000000,545427.000000,528588.000000,528353.000000,525823.00000,524216.000000,524216.000000,517353.000000,517251.000000,515614.000000,515614.000000,515614.000000,515614.000000,512975.000000,512954.000000,512735.000000,510496.000000,508595.000000,508595.000000,508595.000000,508595.000000,508595.000000,508595.000000,508595.000000,508595.000000,508595.000000,508595.000000,508595.000000,508595.000000,508595.000000,508595.000000,508595.000000,508595.000000,508595.000000,508595.000000,508589.000000,508589.000000,508589.000000,508589.000000,508589.000000,508589.000000,508589.000000,508589.000000,508589.000000,508589.000000,508589.000000,508189.00000,508189.00000,508189.00000,508189.00000,508189.00000,508189.00000,508189.00000,508189.00000,508189.00000,508189.00000,508189.00000,508189.00000,508189.00000,508189.00000,508189.00000,508189.00000,508189.00000,508189.00000,471874.000000,463220.000000,461200.00000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,460110.000000,453675.000000,453675.000000,453249.000000,451279.00000,451222.000000,451171.000000,450909.000000,450909.000000,450909.000000,450909.000000,450909.000000,450909.000000,450909.000000,

In [12]:
# Test Data Missing Values
missing_data_train = getnulls(test)
missing_data_train.head(len(test)).T

,id_24,id_25,id_26,id_07,id_08,id_21,id_27,id_22,id_23,dist2,id_18,D7,id_04,id_03,D12,id_30,id_32,id_33,id_14,id_34,id_09,D9,id_10,D8,V153,V157,V154,V155,V156,V158,V148,V161,V162,V163,V149,V147,V146,V138,V139,V140,V141,V142,V166,V164,V160,V159,V143,V144,V145,V152,V165,V150,V151,V328,V327,V339,V338,V337,V336,V335,V334,V333,V332,V331,V330,V322,V323,V324,V325,V329,V326,DeviceInfo,D14,D13,D6,id_16,V257,V268,V277,V276,V275,V274,V273,V247,V248,V249,V269,V265,V267,V266,V246,V264,V263,V262,V261,V260,V252,V258,V278,V242,V244,V243,V217,V218,V219,V223,V224,V225,V226,V228,V229,V230,V231,V232,V233,V235,V236,V237,V240,V241,V254,V253,id_13,id_05,id_06,id_20,R_emaildomain,id_19,id_17,V189,V195,V188,V170,V169,V197,V175,V198,V200,V201,V210,V209,V174,V171,V208,V180,V185,V194,V184,id_31,V199,V206,V187,V205,V204,V203,V202,V190,V191,V186,V176,V192,V181,V167,V196,V168,V179,V178,V177,V172,V173,V193,V182,V183,V207,V212,V213,V214,V215,V216,V211,id_11,id_28,id_29,DeviceType,id_02,id_35,id_15,id_37,id_36,id_38,V255,V234,V256,V238,V239,V222,V245,V250,V251,V221,V220,V272,V271,V270,V259,V227,id_01,id_12,M5,dist1,M4,M7,M8,M9,D2,D5,D3,M3,M1,M2,V1,V2,V5,V6,V7,D11,V11,V10,V9,V3,V4,V8,M6,V40,V41,V42,V45,V44,V52,V51,V50,V49,V48,V47,V46,V43,V39,V38,V37,V36,V35,D4,P_emaildomain,addr1,addr2,V60,V69,V74,V73,V72,V71,V70,V67,V68,V66,V57,V64,V63,V62,V61,V65,V58,V56,V54,V53,V59,V55,V12,V29,V19,V21,V22,V13,V23,V20,V24,V25,V26,V27,V28,V30,V31,V32,V33,V34,V18,V17,V16,V15,V14,D10,V76,V75,V81,V77,V88,V94,V93,V92,V91,V90,V89,V87,V78,V86,V85,V84,V82,V80,V79,V83,D15,card2,D1,V301,V300,V296,V289,V288,V283,V282,V281,V314,V315,V313,C13,card5,card4,card6,card3,C2,C9,V292,V291,V290,C10,V286,V287,V294,V285,V284,V293,V297,V295,C11,C7,V298,V299,C12,V302,V303,V304,V305,C8,V280,C6,C4,V306,C3,V321,V320,V319,V318,C1,V316,V317,C5,C14,V312,V311,V310,V309,V308,V279,V307,TransactionDT,TransactionAmt,ProductCD,card1,V127,V95,V128,V119,V120,V121,V122,V123,V124,V125,V126,V129,V117,V130,V131,V132,V133,V134,V135,V136,V137,V118,V116,V96,V105,V97,V98,V99,V100,V101,V102,V103,V104,V106,V115,V107,V108,V109,V110,V111,V112,V113,V114,TransactionID
total,501951.000000,501652.000000,501644.000000,501632.000000,501632.000000,501632.000000,501629.00000,501629.00000,501629.00000,470255.00000,455816.000000,446558.000000,440210.000000,440210.000000,437437.000000,436032.000000,436020.000000,436020.000000,435334.000000,434516.000000,432353.000000,432353.000000,432353.000000,432353.000000,430906.000000,430906.000000,430906.000000,430906.000000,430906.000000,430906.000000,430906.000000,430906.000000,430906.000000,430906.000000,430906.000000,430906.000000,430906.000000,430906.000000,430906.000000,430906.000000,430906.000000,430906.000000,430636.000000,430636.000000,430636.000000,430636.000000,430636.000000,430636.000000,430636.000000,430636.000000,430636.000000,430636.000000,430636.000000,430260.000000,430260.000000,430260.000000,430260.000000,430260.000000,430260.000000,430260.000000,430260.000000,430260.000000,430260.000000,430260.000000,430260.000000,430260.000000,430260.000000,430260.000000,430260.000000,430260.000000,430260.000000,391634.000000,391497.000000,383307.000000,381908.00000,380944.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,379963.000000,376405.000000,371941.000000,371941.000000,371058.000000,370821.000000,370785.000000,370725.000000,370316.000000,370316.000000,370316.000000,370316.000000,370316.000000,370316.000000,37031

In [13]:
# Get the columns names having too many missing values
droppedcols = missing_data_train[missing_data_train['total'] > 100000].index
droppedcols

Index(['id_24', 'id_25', 'id_26', 'id_07', 'id_08', 'id_21', 'id_27', 'id_22',
       'id_23', 'dist2',
       ...
       'V6', 'V7', 'D11', 'V11', 'V10', 'V9', 'V3', 'V4', 'V8', 'M6'],
      dtype='object', length=233)

In [14]:
len(droppedcols)

233

In [15]:
# Drop the columns
train.drop(droppedcols, axis=1, inplace=True)
test.drop(droppedcols, axis=1, inplace=True)

In [16]:
train['P_emaildomain'].value_counts()

gmail.com           228355
yahoo.com           100934
hotmail.com          45250
anonymous.com        36998
aol.com              28289
comcast.net           7888
icloud.com            6267
outlook.com           5096
msn.com               4092
att.net               4033
live.com              3041
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
yahoo.com.mx          1543
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
live.com.mx            749
rocketmail.com         664
mail.com               559
earthlink.net          514
gmail                  496
outlook.es             438
mac.com                436
juno.com               322
aim.com                315
windstream.net         305
hotmail.es             305
roadrunner.com         305
hotmail.fr             295
frontier.com           280
embarqmail.com         260
web.de                 240
twc.com                230
n

In [17]:
cntgmail = 0
cntyahoo = 0
cnthotmail = 0
cntanon = 0
cntaol = 0
for i in range(len(train)):
    if train['P_emaildomain'][i] == 'gmail.com' and train['isFraud'][i] == 1:
        cntgmail += 1
    elif train['P_emaildomain'][i] == 'yahoo.com' and train['isFraud'][i] == 1:
        cntyahoo += 1
    elif train['P_emaildomain'][i] == 'hotmail.com' and train['isFraud'][i] == 1:
        cnthotmail += 1
    elif train['P_emaildomain'][i] == 'anonymous.com' and train['isFraud'][i] == 1:
        cntanon += 1
    elif train['P_emaildomain'][i] == 'aol.com' and train['isFraud'][i] == 1:
        cntaol += 1
    
print("GMAIL:", cntgmail)
print("YAHOO:", cntyahoo)
print("HOTMAIL:", cnthotmail)
print("ANON:", cntanon)
print("AOL:", cntaol)

GMAIL: 9943
YAHOO: 2297
HOTMAIL: 2396
ANON: 859
AOL: 617


In [18]:
# So any perticular mail doesn't envolve only for fraud, so not related,
#let's drop the column
train.drop(['P_emaildomain'], axis=1, inplace=True)
test.drop(['P_emaildomain'], axis=1, inplace=True)

In [19]:
print(train.shape)
print(test.shape)

(590540, 200)
(506691, 199)


## Concatinating all data to fill the null values with the mean and mode

In [20]:
ntrain = len(train)
ntest = len(test)
all_data = pd.concat([train, test], axis=0, sort=False)
all_data.shape

(1097231, 200)

In [21]:
all_data_cols = all_data.columns
for i in all_data_cols:
    if all_data[i].dtype == 'object':
        all_data[i] = all_data[i].fillna(all_data[i].mode()[0])

In [22]:
for i in all_data_cols:
    if (i.startswith("C") or (i.startswith("V"))) and all_data[i].isnull().sum() > 0:
        all_data[i] = all_data[i].fillna(all_data[i].mode())
missing_data = getnulls(all_data)
missing_data.head(10).T

,isFraud,V47,V41,V51,V50,V49,V48,V46,V45,V44
total,506691.000000,245822.000000,245822.000000,245822.000000,245822.000000,245822.000000,245822.000000,245822.000000,245822.000000,245822.000000
precent,0.461791,0.224039,0.224039,0.224039,0.224039,0.224039,0.224039,0.224039,0.224039,0.224039


In [23]:
all_data['D1'] = all_data['D1'].fillna(all_data['D1'].mode())
all_data['card3'] = all_data['card3'].fillna(all_data['card3'].mean())
all_data['card2'] = all_data['card2'].fillna(all_data['card2'].mean())
all_data['card5'] = all_data['card5'].fillna(all_data['card5'].mean())

In [24]:
missing_data = getnulls(all_data)
missing_data.head(5).T

,isFraud,V36,V43,V52,V35
total,506691.000000,245822.000000,245822.000000,245822.000000,245822.000000
precent,0.461791,0.224039,0.224039,0.224039,0.224039


## All the null values have been handled

In [25]:
# Split the data again
train = all_data[:ntrain]
test = all_data[ntrain:]

In [26]:
print(train.shape)
print(test.shape)

(590540, 200)
(506691, 200)


In [27]:
print(test['isFraud'].value_counts())
test.drop(['isFraud'], axis=1, inplace=True)
print(test.shape)

Series([], Name: isFraud, dtype: int64)
(506691, 199)


## Concatinating the data again to encode the categorical features

In [28]:
ntrain = train.shape[0]
ntest = test.shape[0]
all_data = pd.concat([train, test], axis=0, sort=False)
print(all_data.shape)

del train,test

(1097231, 200)


In [29]:
# Encoding the data
all_data = pd.get_dummies(all_data)
print(all_data.shape)
all_data.head()

(1097231, 210)


,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D4,D10,D15,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,card4_american express,card4_discover,card4_mastercard,card4_visa,card6_charge card,card6_credit,card6_debit,card6_debit or credit
0,2987000,0.0,86400,68.5,13926,363.099769,150.0,142.0,315.0,87.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,1,0,0,0,1,0,0
1,2987001,0.0,86401,29.0,2755,404.000000,150.0,102.0,325.0,87.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,1,0,0,1,0,0
2,2987002,0.0,86469,59.0,4663,490.000000,150.0,166.0,330.0,87.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,315.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,1,0,0,1,0
3,2987003,0.0,86499,50.0,18132,567.000000,150.0,117.0,476.0,87.0,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,94.0,84.0,111.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,

In [30]:
# Split the data again and get ready for training
train = all_data[:ntrain]
test = all_data[ntrain:]


In [31]:
train.drop(['TransactionID'], axis=1, inplace=True)
test_id = test['TransactionID']
test.drop(['TransactionID'], axis=1, inplace=True)
print(train.shape)
print(test.shape)

In [32]:
train.drop(['isFraud'], axis=1, inplace=True)
test.drop(['isFraud'], axis=1, inplace=True)
print(train.shape)
print(test.shape)

In [33]:
%%time
# XGBOOST
xgmodel = xgb.XGBClassifier(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=50, 
                             min_child_weight=1.7817, n_estimators=200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, random_state =7, nthread = -1)
xgmodel.fit(train, target)